In [ ]:
!pip install pickleshare=0.7.5
!pip install numpy=1.20.2
!pip install tqdm=4.62.2
!pip install torch=1.8.1
!pip install tensorboardx=2.4
!pip install scipy=1.7.0
!pip install lripy=0.0.2
!pip install torchvision=0.9.1

In [ ]:
!wget https://modelarts-cnnorth1-market-dataset.obs.cn-north-1.myhuaweicloud.com/example-apps/FedVal/code.zip
!unzip -qo code.zip 

In [27]:
import os
import copy
import time
import pickle
import numpy as np
from tqdm import tqdm
import torch
from tensorboardX import SummaryWriter
from options import args_parser
from update import LocalUpdate, test_inference
from models import MLP, CNNCifar2, CNNMnist, CNNFashion_Mnist, VGG16, Logistic
from utils import get_dataset, average_weights, exp_details
from itertools import chain, combinations
import operator as op
from functools import reduce
import sys
from scipy.sparse import csr_matrix
import pdb
from lripy import drcomplete
from numpy import linalg as LA
from torchvision import datasets, transforms
from sampling import mnist_iid, mnist_noniid, mnist_noniid_unequal
from sampling import cifar_iid, cifar_noniid

In [2]:
# function for generating the power set
def powerset(iterable):
    s = list(iterable)
    l = chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))
    return {tuple(sorted(tmp)):i for i,tmp in enumerate(l)}

In [3]:
# function for computing the combinatorial number
def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom

In [4]:
# roundly utility function
def utility(args, previous_loss, fake_model, weights, test_dataset):
    fake_weights = average_weights(weights)
    fake_model.load_state_dict(fake_weights)
    _, fake_test_loss = test_inference(args, fake_model, test_dataset)
    return previous_loss - fake_test_loss

In [5]:
# compute all utilities for selected clients
def compute_utilities(args, previous_loss, fake_model, all_subsets, local_weights, test_dataset, idxs_users):
    utilities = np.zeros( len(all_subsets) )
    utilities_dict = {}
    for i, indices in enumerate(tqdm(all_subsets.keys())):
        weights = [local_weights[j] for j in indices]
        u = utility(args, previous_loss, fake_model, weights, test_dataset)
        utilities[i] = u
        utilities_dict[indices] = u
    return utilities, utilities_dict

In [6]:
# baseline
def compute_shapley_value_baseline(args, utilities_dict, idxs_users):
    N = len(idxs_users)
    roundly_valuation_baseline = np.zeros(args.num_users)
    for i in range( len(idxs_users) ):
        tmp_indices = list(idxs_users)
        current_i = tmp_indices.pop(i)
        subpowerset = powerset(tmp_indices)
        val = 0
        for s in subpowerset.keys():
            si=tuple(sorted(list(s)+[current_i]))
            val += (utilities_dict[si]-utilities_dict[s]) / ncr(N - 1, len(s))
        roundly_valuation_baseline[current_i] = val / N
    return roundly_valuation_baseline

In [7]:
# ground truth
def compute_shapley_value_groundtruth(args, utilities_dict):
    N = args.num_users
    roundly_valuation_groundtruth = np.zeros(N)
    for i in range( N ):
        tmp_indices = list(range(N))
        current_i = tmp_indices.pop(i)
        subpowerset = powerset(tmp_indices)
        val = 0
        for s in subpowerset.keys():
            si=tuple(sorted(list(s)+[current_i]))
            val += (utilities_dict[si]-utilities_dict[s]) / ncr(N - 1, len(s))
        roundly_valuation_groundtruth[current_i] = val / N
    return roundly_valuation_groundtruth

In [8]:
# mask in each round
def roundly_mask(idxs_users, all_subsets):
    mask_vec = np.zeros( len(all_subsets) )
    subpowerset = powerset(idxs_users)
    for s in subpowerset.keys():
        i = all_subsets[s]
        mask_vec[i] = 1
    return mask_vec

In [9]:
# our approach
def compute_shapley_value_from_matrix(args, utility_matrix, all_subsets):
    T = args.epochs
    N = args.num_users
    valuation_completed = np.zeros(N)
    for i in range(N):
        sublist = list(range(N))
        sublist.pop(i)
        subpowerset = powerset(sublist)
        for s in subpowerset.keys():
            # id1 = all_subsets.index(s)
            id1 = all_subsets[s]
            id2 = all_subsets[tuple(sorted(list(s)+[i]))]
            for t in range(T):
                v1 = utility_matrix[t, id1]
                v2 = utility_matrix[t, id2]
                val = (v2 - v1) / ncr(N - 1, len(s))
                valuation_completed[i] += val
        valuation_completed[i] /= N
    return valuation_completed

In [10]:
class Arguments:
    def __init__(self):
        self.epochs = 10
        self.num_users = 10
        self.frac = 0.5
        self.local_ep = 1
        self.local_bs = 50
        self.lr = 0.01
        self.momentum = 0.0
        self.optimizer = 'sgd'
        self.gpu = 1
        self.verbose = 0

In [11]:
args = Arguments()

In [12]:
# load data
data_dir = '../data/mnist/'
apply_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(data_dir, train=True, download=True,
                                       transform=apply_transform)
test_dataset = datasets.MNIST(data_dir, train=False, download=True,
                                      transform=apply_transform)
test_dataset.data = test_dataset.data[:1000]
test_dataset.targets = test_dataset.targets[:1000]

In [13]:
# split data in a non-iid or iid fashion
# client 0 and 9 have same data
user_groups = mnist_noniid(train_dataset, args.num_users)
# user_groups = mnist_iid(train_dataset, args.num_users)

/home/zhenan/miniconda3/envs/fedval/lib/python3.7/site-packages/torchvision/datasets/mnist.py:54: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [14]:
# build MLP model
img_size = train_dataset[0][0].shape
len_in = 1
for x in img_size:
    len_in *= x
#     global_model=Logistic(in_dim=len_in, out_dim=10)
global_model = MLP(dim_in=len_in, dim_hidden=64,
                       dim_out=10)
device = 'cuda'
global_model.to(device)
global_model.train()
global_weights = global_model.state_dict()
print(global_model)

MLP(
  (layer_input): Linear(in_features=784, out_features=64, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_hidden): Linear(in_features=64, out_features=10, bias=True)
  (logsoftmax): LogSoftmax(dim=1)
)


In [15]:
# Initialization
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 1
val_loss_pre, counter = 0, 0
valuation_baseline = np.zeros(args.num_users)
valuation_groundtruth = np.zeros(args.num_users)
all_subsets = powerset(range(args.num_users))
utility_matrix = np.zeros( (args.epochs, len(all_subsets.keys())) )
mask = np.zeros( (args.epochs, len(all_subsets.keys())) )
previous_loss = 0.0
fake_model=copy.deepcopy(global_model)
# define paths
path_project = os.path.abspath('..')
logger = SummaryWriter('../logs')

In [16]:
# main loop of training
for epoch in range(args.epochs):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')
    global_model.train()
    
    # select participants
    m = max(int(args.frac * args.num_users), 1)
    if epoch == 0:
        idxs_users = list(range(args.num_users))
    else:
        idxs_users = np.random.choice(range(args.num_users), m, replace=False)
        
    # train local models
    for idx in range(args.num_users):
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                    idxs=user_groups[idx], logger=logger, client_idx=idx, is_noisy=False)
        if idx==args.num_users-1:
            w = copy.deepcopy(local_weights[0])
            loss = copy.deepcopy(local_losses[0])
        else:
            w, loss = local_model.update_weights(model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))
        
    # compute shapley values
    utilities, utilities_dict = compute_utilities(args, previous_loss, fake_model, 
                                                  all_subsets, local_weights, test_dataset, idxs_users)
    utility_matrix[epoch, :] = utilities
    mask_epoch = roundly_mask(idxs_users, all_subsets)
    mask[epoch, :] = mask_epoch
    valuation_baseline_epoch = compute_shapley_value_baseline(args, utilities_dict, idxs_users)
    valuation_baseline += valuation_baseline_epoch
    valuation_groundtruth_epoch = compute_shapley_value_groundtruth(args, utilities_dict)
    valuation_groundtruth += valuation_groundtruth_epoch
    
    # update global weights
    local_weights_selected = [local_weights[i] for i in list(idxs_users)]
    global_weights = average_weights(local_weights_selected)
    global_model.load_state_dict(global_weights)
    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)
    
    # Calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    global_model.eval()
    for c in range(args.num_users):
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger, client_idx=c, is_noisy=False)
        acc, loss = local_model.inference(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
    train_accuracy.append(sum(list_acc)/len(list_acc))
    
    # test loss 
    test_acc, test_loss = test_inference(args, global_model, test_dataset)
    previous_loss = test_loss
    
    # print training and test losses after every 'i' rounds
    if (epoch+1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch+1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))


 | Global Training Round : 1 |



/home/zhenan/Federated-Learning-PyTorch/src/update.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)
100%|█████████████████████████████████████████████████████████| 1023/1023 [01:44<00:00,  9.77it/s]


 
Avg Training Stats after 1 global rounds:
Training Loss : 1.6753390914201738
Train Accuracy: 1.67% 


 | Global Training Round : 2 |



100%|█████████████████████████████████████████████████████████| 1023/1023 [01:45<00:00,  9.66it/s]


 
Avg Training Stats after 2 global rounds:
Training Loss : 1.5184518268704414
Train Accuracy: 96.67% 


 | Global Training Round : 3 |



...

 
Avg Training Stats after 8 global rounds:
Training Loss : 1.077837294526398
Train Accuracy: 80.00% 


 | Global Training Round : 9 |



100%|█████████████████████████████████████████████████████████| 1023/1023 [01:44<00:00,  9.75it/s]


 
Avg Training Stats after 9 global rounds:
Training Loss : 1.0380750592384074
Train Accuracy: 88.33% 


 | Global Training Round : 10 |



100%|█████████████████████████████████████████████████████████| 1023/1023 [01:44<00:00,  9.78it/s]


 
Avg Training Stats after 10 global rounds:
Training Loss : 1.0063412492573263
Train Accuracy: 91.67% 



In [22]:
# complete matrix
mask = mask.astype(int)
U = csr_matrix(np.multiply(utility_matrix, mask))
utility_matrix_complete = drcomplete(utility_matrix, mask, 3, 2)[0]

Error between X_281 and Y_281 <= 0.1
Error between X_511 and Y_511 <= 0.01
Error between X_1211 and Y_1211 <= 0.001
Error between X_2555 and Y_2555 <= 0.0001
Error between X_3899 and Y_3899 <= 1e-05


In [23]:
# compute ComFedSV
valuation_completed = compute_shapley_value_from_matrix(args, utility_matrix_complete, all_subsets)

In [28]:
# normalization
sb = sum(valuation_baseline) 
valuation_baseline = [v/sb for v in valuation_baseline]
sg = sum(valuation_groundtruth) 
valuation_groundtruth = [v/sg for v in valuation_groundtruth]
sc = sum(valuation_completed) 
valuation_completed = [v/sc for v in valuation_completed]

In [32]:
# print the valuation difference between client 0 and client 9 for all three metrics
print("relative diff(FedSV) = ", 
      abs(valuation_baseline[0]-valuation_baseline[9])/min(valuation_baseline[0],valuation_baseline[9]))
print("relative diff(ComFedSV) = ", 
      abs(valuation_completed[0]-valuation_completed[9])/min(valuation_completed[0],valuation_completed[9]))
print("relative diff(Ground-truth) = ", 
      abs(valuation_groundtruth[0]-valuation_groundtruth[9])/min(valuation_groundtruth[0],valuation_groundtruth[9]))

relative diff(FedSV) =  0.09489009013302833
relative diff(ComFedSV) =  5.745064990122422e-05
relative diff(Ground-truth) =  2.821971767457566e-08
